# Analysing steps in a short walk using acceleration and rotation

In this noteboook we examine a recording from the Arduino IMU of a short walk with the aim to extract the step count, cadence and timings including ground time and step duration.

We implement three techniques outlined in the paper `A comprehensive comparison of simple step counting techniques using wrist- and ankle-mounted accelerometer and gyroscope signals` by Matthew Rudy and Joseph Mahoney - [https://www.researchgate.net/publication/325451208_A_comprehensive_comparison_of_simple_step_counting_techniques_using_wrist-_and_ankle-mounted_accelerometer_and_gyroscope_signals](https://www.researchgate.net/publication/325451208_A_comprehensive_comparison_of_simple_step_counting_techniques_using_wrist-_and_ankle-mounted_accelerometer_and_gyroscope_signals).

* Peak-finding
* Fast Fourier Transform (FFT)
* Autocorrelation

Each of these methods allows us to count steps. The peak-finding method also identifies where the steps occur in the timeseries, so this in turn allows us to isolate steps and calculate such things as ground time and step duration.

## The IMU

The Arduino Nano inertial measurement unit gives us acceleration, measured in `g`s (`1g = 9.8m/s/s`), and rotation (angular velocity). In this notebook we demonstrate how to extract steps from either acceleration or rotation.

## The data

We expect a CSV file with columns for time, 3 axes of acceleration, and 3 axes of gyroscopic rotation.

## Setup

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.integrate import cumtrapz
from scipy.signal import butter, filtfilt, periodogram, spectrogram, find_peaks
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11, 4)})
import numpy as np

## Data import

In [ ]:
data_file_path = "../data/slow-walking.csv"
df = pd.read_csv(data_file_path, header=None)
df.columns = ["time", "aX", "aY", "aZ", "gX", "gY", "gZ"]

## Key parameters

In [ ]:
# params
fs = 1000 / 100 # Hz, sampling frequency
total_time = df.time.max() - df.time.min()

## Helper functions

In [ ]:
def low_pass(data, fc, fs):
    w = fc / (fs / 2) # Normalize the frequency
    b, a = butter(5, w, 'low')
    return filtfilt(b, a, data)

def get_magnitude(data, cutoff_frequency=None, fs=None):
    magnitude = np.sqrt((data**2).sum(axis=1))
    if cutoff_frequency is None:
        return magnitude
    else:
        return low_pass(magnitude, cutoff_frequency, fs)

def peak_detection_steps(data, pos_kwargs=None, neg_kwargs=None):
    peaks, _ = find_peaks(data, **pos_kwargs)
    neg_peaks, _ = find_peaks(-data, **neg_kwargs)
    sns.lineplot(x=range(len(data)), y=data)
    sns.scatterplot(x=peaks, y=data[peaks])
    sns.scatterplot(x=neg_peaks, y=data[neg_peaks])
    plt.show()
    return len(peaks)

def fft_steps(data, dt, fs):
    f, Pxx = periodogram(data, fs=fs)
    sns.lineplot(f, Pxx)
    plt.show()
    return dt * f[np.argmax(Pxx)] / 1000

def autocorr(x):
    result = np.correlate(x, x, mode='same')
    return result[:]

def autocorr_steps(data):
    corr = autocorr(data)
    sns.lineplot(x=range(len(corr)), y=corr)
    peaks, _ = find_peaks(corr)
    sns.scatterplot(x=peaks, y=corr[peaks])
    plt.show()
    return len(peaks)

# Measuring steps from acceleration

In [ ]:
sns.lineplot(x=df.time, y=df.aX, label="aX")
sns.lineplot(x=df.time, y=df.aY, label="aY")
sns.lineplot(x=df.time, y=df.aZ, label="aZ")

In [ ]:
_df = df.iloc[:9]
sns.lineplot(x=_df.time, y=_df.aX, label="aX")
sns.lineplot(x=_df.time, y=_df.aY, label="aY")
sns.lineplot(x=_df.time, y=_df.aZ, label="aZ")

In [ ]:
gravity_vector = _df.loc[:,["aX", "aY", "aZ"]].mean()
(gravity_vector, (gravity_vector**2).sum())

In [ ]:
df.loc[:,["aX", "aY", "aZ"]] = (df.loc[:,["aX", "aY", "aZ"]] - gravity_vector) * 9.8

In [ ]:
aMagnitude = get_magnitude(df.loc[:,["aX", "aY", "aZ"]], cutoff_frequency=1.5, fs=fs)
n_peak_steps = peak_detection_steps(
    aMagnitude, 
    pos_kwargs={
        "prominence": 2,
        "distance": 3
    }, 
    neg_kwargs={
        "prominence": 1
    }
)
n_fft_steps = fft_steps(aMagnitude, dt=total_time, fs=fs)
n_autocorr_steps = autocorr_steps(aMagnitude)
(n_peak_steps, n_fft_steps, n_autocorr_steps)

# Measuring steps from rotation

In [ ]:
sns.lineplot(x=df.time, y=df.gX, label="gX")
sns.lineplot(x=df.time, y=df.gY, label="gY")
sns.lineplot(x=df.time, y=df.gZ, label="gZ")

In [ ]:
gMagnitude = get_magnitude(df.loc[:,["gX", "gY", "gZ"]], cutoff_frequency=1.5, fs=fs)
n_peak_steps = peak_detection_steps(
    gMagnitude, 
    pos_kwargs={
        "prominence": 2,
        "distance": 3
    }, 
    neg_kwargs={
        "prominence": 1
    }
)
n_fft_steps = fft_steps(gMagnitude, dt=total_time, fs=fs)
n_autocorr_steps = autocorr_steps(gMagnitude)
(n_peak_steps, n_fft_steps, n_autocorr_steps)